# Combined analysis / Heatmap generation

### Extract neuron.S

In [1]:
# Run csvwrite('ca.csv', full(neuron.S)) in MATLAB

### Build the frameMap from the timestamp.dat

In [ ]:
build_frame_map(r'TIMESTAMP_DAT_PATH', r'WORKING_FOLDER')

### Transform coordinates

In [ ]:
transform_coordinates(r'H5_PATH', np.float32([[TOP,LEFT], [TOP,RIGHT], [BOTTOM,RIGHT], [BOTTOM,LEFT]]), 44, 44,
                      r'WORKING_FOLDER')

### Generate heatmaps

In [ ]:
# Enclosure length: 10 cm; Width: 4.5 cm
generate_heatmaps(r'WORKING_FOLDER', np.float32(((17,39.5), (27,39.5), (27,44), (17,44))), int(sqrt(27**2 + 44**2)))

## *Run all below before calling them*

### Build the frameMap from the timestamp.dat

In [ ]:
import os
import csv

In [ ]:
def build_frame_map(time_stamp_path, work_fold):
    frame_map, time_stamps = [], []
    with open(time_stamp_path) as time_stamp_file:
        _ = csv.reader(time_stamp_file, delimiter='\t')
        for i in _:
            time_stamps.append(i)

    print('First 5 lines of the timestamp.dat:')
    for i in time_stamps[:5]:
        print (*i, sep='\t')
    print('Last 5 lines:')
    for i in time_stamps[-5:]:
        print (*i, sep='\t')
    ca_cam = time_stamps[1][0]
    beh_cam = time_stamps[2][0]
    print('Decide the camNum of Ca2+ imaging:\n(Normally, the one with larger frameNum at the end)')
    if (input(ca_cam + '/' + beh_cam) != ca_cam):
        ca_cam, beh_cam = beh_cam, ca_cam

    ca_count, beh_count = 0, 0
    last_beh_frame = '-1'
    for i in time_stamps[1:]:
        if (i[0] == ca_cam):
            ca_count += 1
        else:
            while(beh_count < ca_count):
                frame_map.append(i[1])
                beh_count += 1
            last_beh_frame = i[1]

    with open(Path(work_fold)/'ca2beh.csv', 'w', newline='') as frame_map_csv_file:
        frame_map_csv_writ = csv.writer(frame_map_csv_file)
        for i in frame_map:
            frame_map_csv_writ.writerow([i])
    print('A Frame Map (Map of behaviour frames with respect to the Ca2+ imaging frames) has been built as ca2beh.csv in ' + work_fold)

### Transform coordinates

In [ ]:
import numpy as np
from pathlib import Path
import pandas as pd
import cv2

In [ ]:
# Calculate transformation matrix
def clc_trnsf_mat(src_qdr_vrts, dst_wdth, dst_hght):
    dst_qdr_vrts = np.float32([[0, 0],
                               [dst_wdth-1, 0],
                               [dst_wdth-1, dst_hght-1],
                               [0, dst_hght-1]])
    return cv2.getPerspectiveTransform(src_qdr_vrts, dst_qdr_vrts)

In [ ]:
# Transform points
def trnsf_pnts(src_pnts, src_qdr_vrts, dst_wdth, dst_hght):
    # Convert 2D (x, y) to 3D (x, y, 1)
    src_pnts = np.concatenate((src_pnts, np.ones((src_pnts.shape[0], 1))), axis=1)

    # Transformation matrix
    trnsf_mat = clc_trnsf_mat(src_qdr_vrts, dst_wdth, dst_hght)
    # Since (destination points)^T = (transformation matrix)(source points)^T
    # And (AB)^T = B^TA^T
    # Destination points = (source points)(transformation matrix)^T
    dst_pnts = np.dot(src_pnts, trnsf_mat.T)

    return dst_pnts[:,:2]

In [ ]:
# src_qdr_vrts: Source quadrangle vertices (top-left, top-right, bottom-right, and bottom-left)
# dst_wdth: Destination width
# dst_hght: Destination height
def transform_coordinates(h5_path, src_qdr_vrts, dst_wdth, dst_hght, work_fold):
    csv_path = Path(h5_path).parent/(Path(h5_path).stem+'.csv')
    
    
    # Converse .h5 to .csv
    h5 = pd.read_hdf(h5_path, 'df_with_missing')
    h5.to_csv(csv_path)
    
    
    # Read .csv
    csv = np.loadtxt(csv_path, delimiter=',', skiprows=3)

    src_heads, src_bods, src_tails = csv[:,1:3], csv[:,4:6], csv[:,7:9]

    dst_heads = trnsf_pnts(src_heads, src_qdr_vrts, dst_wdth, dst_hght)
    dst_bods = trnsf_pnts(src_bods, src_qdr_vrts, dst_wdth, dst_hght)
    dst_tails = trnsf_pnts(src_tails, src_qdr_vrts, dst_wdth, dst_hght)

    dst = np.concatenate((dst_heads, csv[:,3:4], dst_bods, csv[:,6:7], dst_tails, csv[:,9:10]), axis=1)
    np.savetxt(Path(work_fold)/'beh.csv', dst, delimiter=',')

### Generate heatmaps

In [ ]:
import os
from math import sqrt, atan2, cos, sin, degrees
import csv
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Circle-line intersection: http://mathworld.wolfram.com/Circle-LineIntersection.html
def sgn(x):
    return -1 if x < 0 else 1

def int_circ_line(r, x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    dr = sqrt(dx**2 + dy**2)
    d = x1*y2 - x2*y1
    
    delta = r**2 * dr**2 - d**2
    if delta < 0:
        return []
    
    x, y = [], []
    _x = (d*dy + sgn(dy)*dx*sqrt(delta)) / dr**2
    _y = (-d*dx + abs(dy)*sqrt(delta)) / dr**2
    # If the point on the line falls within the segment
    if ((_x-x1) * (_x-x2) < 0 or
        (_y-y1) * (_y-y2) < 0):
        x.append(_x)
        y.append(_y)
    
    _x = (d*dy - sgn(dy)*dx*sqrt(delta)) / dr**2
    _y = (-d*dx - abs(dy)*sqrt(delta)) / dr**2
    if ((_x-x1) * (_x-x2) < 0 or
        (_y-y1) * (_y-y2) < 0):
        x.append(_x)
        y.append(_y)
    
    # Theta = tan^-1(y / x)
    theta = [atan2(_y, _x) for _x, _y in zip(x, y)]
    return theta

In [ ]:
# Check if a point, given its polar coordinates, is inside a rectangle
def is_ins_encl(r, theta, encl_qdr_vrts): 
    # M is inside the rectangle iff
    # (0 < AM dot AB < AB dot AB) and (0 < AM dot AD < AD dot AD)
    # See: https://math.stackexchange.com/questions/190111/how-to-check-if-a-point-is-inside-a-rectangle
    m = np.float32((r*cos(theta), r*sin(theta)))
    a = encl_qdr_vrts[0]
    b = encl_qdr_vrts[1]
    d = encl_qdr_vrts[3]
    
    am = m-a
    ab = b-a
    ad = d-a
    
    return ((0 < np.dot(am, ab) and np.dot(am, ab) < np.dot(ab, ab)) and
            (0 < np.dot(am, ad) and np.dot(am, ad) < np.dot(ad, ad)))

In [ ]:
# src_encl_qdr_vrts: Source enclosure quadrangle vertices (top-left, top-right, bottom-right, and bottom-left)
# htm_rad: Heatmap radius
def generate_heatmaps(work_fold, src_encl_qdr_vrts, htm_rad):
    curr_dir = os.getcwd()
    os.chdir(work_fold)

    # Epsilon
    eps = np.finfo(np.float32).eps
    
    
    heads, bods = [], []
    with open('beh.csv') as _f:
    _c = csv.reader(_f)
    for i in _c:
        heads.append([float(_) for _ in i[0:2]])
        bods.append([float(_) for _ in i[3:5]])
        
        
    ca = np.loadtxt('ca.csv', delimiter=',')

    ca2beh = []
    with open('ca2beh.csv') as _f:
        _c = csv.reader(_f)
        for i in _c:
            ca2beh.append(int(i[0]))
            
            
    dst_encl_qdr_vrts = []
    for i, j in zip(heads, bods):
        # Change of basis: https://www.math.hmc.edu/calculus/tutorials/changebasis/
        u = np.float32((i[0]-j[0], i[1]-j[1]))
        u = u / sqrt(u[0]**2 + u[1]**2)
        # Perpendicular vector: http://mathworld.wolfram.com/PerpendicularVector.html
        # Notice that the coordinate system is being changed from left-handed to right-handed
        # Hence (-u[1], u[0]) becomes (u[1], -u[0])
        v = np.float32((u[1], -u[0]))
        p = np.stack((u,v), axis=1)
    
        # Destination enclosure quadrangle vertices
        _ = src_encl_qdr_vrts - np.float32((i[0], i[1]))
        # Since (destination enclosure quadrangle vertices)^T = p^T(source enclosure quadrangle vertices)^T
        # And (AB)^T = B^TA^T
        # Destination enclosure quadrangle vertices = (source enclosure quadrangle vertices)p
        _ = np.dot(_, p)
        dst_encl_qdr_vrts.append(_)
        
        
    # Arcs of the concentric circles covered by each enclosure
    arcs_encl = []
    for i in dst_encl_qdr_vrts:
        # Enclosure-covered arcs of each concentric circles
        arcs_circ = []
        for j in range(1, htm_rad+1):
            theta = []
            for k in range(4):
                theta += int_circ_line(j, i[k][0], i[k][1], i[(k+1)%4][0], i[(k+1)%4][1])

            if len(theta) < 2:
                arcs_circ.append([])
                continue
        
            theta = sorted(theta)
        
            if is_ins_encl(j, (theta[0]+theta[1])/2, i):
                theta = [degrees(_) for _ in theta]
                theta = [int(_) for _ in theta]
                theta = [_+360 if _ < 0 else _ for _ in theta]
                theta = [theta[_:_+2] for _ in range(0, len(theta)-1, 2)]
            else:
                theta = [degrees(_) for _ in theta]
                theta = [int(_) for _ in theta]
                theta = [_+360 if _ < 0 else _ for _ in theta]
                theta.append(theta[0])
                theta = [theta[_:_+2] for _ in range(1, len(theta), 2)]
            arcs_circ.append(theta)
        
        arcs_encl.append(arcs_circ)
        
        
    # Occupancy heatmap
    occ = np.zeros((htm_rad, 360), dtype='float32')
    for i in arcs_encl:
        for j in range(htm_rad):
            for k in i[j]:
                if k[0] <= k[1]:
                    for l in range(k[0], k[1]):
                        occ[j][l] += 1
                else:
                    for l in range(k[0], 360):
                        occ[j][l] += 1
                    for l in range(0, k[1]):
                        occ[j][l] += 1

    # Adapted from: https://www.pythonprogramming.in/how-do-i-create-radial-heatmap-in-matplotlib.html
    fig = plt.figure()
    ax = Axes3D(fig)

    rad = np.linspace(0, htm_rad, htm_rad+1)
    a = np.linspace(0, 2*np.pi, 361)
    r, th = np.meshgrid(rad, a)

    z = occ.T
    plt.subplot(projection="polar")
 
    plt.pcolormesh(th, r, z, cmap='coolwarm')
 
    plt.plot(a, r, ls='None') 
    plt.grid()
    plt.colorbar(ticks=[])

    plt.savefig('occ.png', dpi=300)
    plt.show()
    
    
    # Firing heatmaps for individual cells
    for i in range(ca.shape[0]):
        fir = np.zeros((htm_rad, 360), dtype='float32')
    
        for j in range(min(ca[i].shape[0], len(ca2beh))):
            if ca[i][j] < eps:
                continue
        
            _ = ca[i][j]
            for k in range(htm_rad):
                for l in arcs_encl[ca2beh[j]-1][k]:
                    if l[0] <= l[1]:
                        for m in range(l[0], l[1]):
                            fir[k][m] += _
                    else:
                        for m in range(l[0], 360):
                            fir[k][m] += _
                        for m in range(0, l[1]):
                            fir[k][m] += _
        
        fir /= occ
        
        # Adapted from: https://www.pythonprogramming.in/how-do-i-create-radial-heatmap-in-matplotlib.html
        fig = plt.figure()
        ax = Axes3D(fig)

        rad = np.linspace(0, htm_rad, htm_rad+1)
        a = np.linspace(0, 2*np.pi, 361)
        r, th = np.meshgrid(rad, a)

        z = fir.T
        plt.subplot(projection="polar")
 
        plt.pcolormesh(th, r, z, cmap='coolwarm')
 
        plt.plot(a, r, ls='None')
        plt.grid()
        plt.colorbar(ticks=[])
    
        plt.savefig(str(i+1)+'.png', dpi=300)
        plt.show()
        
        
    os.chdir(curr_dir)